In [420]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [421]:
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/hist_data.csv')

In [422]:
hist_data.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [423]:
morning_orders, day_orders, evening_orders = 0, 0, 0

In [424]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4529889 entries, 0 to 4529888
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   buyer_id           int64  
 1   pav_order_id       int64  
 2   created            object 
 3   item_id            int64  
 4   count              float64
 5   price_sold         float64
 6   flag_weight_goods  bool   
 7   weight             float64
dtypes: bool(1), float64(3), int64(3), object(1)
memory usage: 246.2+ MB


In [425]:
!free -g

              total        used        free      shared  buff/cache   available
Mem:            503         301         176           0          25         199
Swap:             7           7           0


In [426]:
train.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [427]:
popular_by_user = train.groupby('buyer_id')['item_id'].agg(lambda x: [x[0] for x in Counter(x).most_common(20)])
popular_by_user.head(5)

buyer_id
94578442    [202979560, 203438161, 203529348, 203457496, 2...
94578475    [203114041, 202820148, 203884509, 202791192, 2...
94578533    [203446790, 202809628, 202864758, 202872237, 2...
94578595    [202801849, 202799674, 202806977, 202997601, 2...
94578656    [202838754, 202872237, 202910396, 202820148, 2...
Name: item_id, dtype: object

In [428]:
known_users = (set(train['buyer_id']) & set(test['buyer_id']))
len(known_users)

63925

In [429]:
len(test['buyer_id'].unique())

80244

In [430]:
order_by_user = test.groupby('pav_order_id')['buyer_id'].apply(lambda x: list(x)[0])
len(order_by_user) == len(test['pav_order_id'].unique())

True

In [431]:
pred = pd.read_csv('data/sample_sab_inverse_after5_true.csv',
                   index_col=0,
                   converters={"preds": lambda x: x.strip("[]").replace("'","").split(", ")})

In [432]:
pred['preds'] = pred['preds'].apply(lambda x: [int(i) for i in x])

In [433]:
# [i for i in pred['preds']]

In [434]:
pred['buyer_id'] = order_by_user[pred.index]
pred.head()

,preds,buyer_id
pav_order_id,,
4620121489,"[202820148, 202872237, 202791620, 202809628, 2...",95969605
4620121505,"[202820148, 202872237, 202880262, 203068900, 2...",95823859
4620121594,"[202820148, 202872237, 203059303, 202809628, 2...",95487550
4620121684,"[202820148, 202872237, 203090014, 203090010, 2...",95970193
4620121902,"[202820148, 203422957, 203431923, 202872237, 2...",95803805


In [435]:
popular_by_user.iloc[0][:1]

[202979560]

In [441]:
def modify_row(row, k=5, m=5):
  if row['buyer_id'] in known_users:
    row['preds'] = row['preds'][:5] + row['preds'][5::-1]
    row['preds'] = row['preds'][:m] + row['preds'][m::-1]
    row['preds'] = set(popular_by_user[row['buyer_id']][:k] + (row['preds'])[:-k])
    row['preds'] = (list(row['preds']) + popular_by_user[row['buyer_id']][k:])[:20]
  return row

In [442]:
new_pred = pred.apply(modify_row, axis=1)
new_pred.drop(['buyer_id'], axis=1, inplace=True)
new_pred.head()

,preds
pav_order_id,
4620121489,"[202820148, 202872237, 202791620, 202809628, 2..."
4620121505,"[202820148, 202872237, 202880262, 203068900, 2..."
4620121594,"[202820148, 202872237, 203059303, 202809628, 2..."
4620121684,"[202820148, 202872237, 203090014, 203090010, 2..."
4620121902,"[202820148, 203422957, 203431923, 202872237, 2..."


In [ ]:
# new_pred.drop(['basket'], axis=1, inplace=True)

In [443]:
new_pred['preds'].apply(len).max()

20

In [444]:
new_pred['preds'].to_csv('data/fixed_v30.csv')

In [68]:
# Пример решения с использованием статистического подхода - подсчет 
# совстречаемостей.

import pandas as pd
import numpy as np
import gc

from collections import Counter 

hist_data = pd.read_csv('data/hist_data.csv')

# соберем словарь встречаемостей - какие item_id покупались чаще с 
# каждым item_id 
tmp = (
    hist_data[['item_id', 'pav_order_id']]
    .sort_values(['item_id', 'pav_order_id'])
    .merge(hist_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
)
tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))

most_freq_dict = {k: v for (k, v) in tmp1.iteritems()}

del tmp1, tmp
gc.collect()

test = pd.read_csv('data/test.csv')

# из списка кандидатов по совстречаемости удаляем повторяющиеся item_id, 
# сохраняя порядок
def get_unique_recs(recs: list, top_n: int) -> list:
    rec_dict = {}
    counter = 0
    for k, v in recs:
        if k not in rec_dict:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break
    return list(rec_dict.keys())

def rec_by_item(item_id: int, most_freq_dict: dict) -> list:
    
    return most_freq_dict.get(item_id, None)

# для каждого item_id соберем top_n самых часто встречающихся item_id, 
# отсортируем по частоте и выберем уникальные
def rec_by_basket(basket: list, most_freq_dict: dict, top_n: int = 20) -> list:
    
    res = []
    for item in basket:
        recs = rec_by_item(item, most_freq_dict)
        if recs is not None:
            res += recs
    
    res = sorted(res, key=lambda x: x[1], reverse=True)
    
    return get_unique_recs(res, top_n)

pred = test.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict))
# pred['preds'] =[i[:5] + i[5:][::-1] for i in pred['preds']]

# pred['preds'].to_csv('sample_sab_inverse_after5.csv')